In [2]:
%load_ext pycodestyle_magic


In [21]:
all_indices_to_consider = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
param_distributions = {
    'standard_scaler__column_indices_to_replace': [all_indices_to_consider],   
    'min_max_scaler__column_indices_to_replace': [all_indices_to_consider]
}
for key, value in param_distributions.items():
        class_key, parameter_name = key.split("__")
        setattr(pipe.named_steps[class_key], parameter_name, value)
        print(pipe.named_steps[class_key], parameter_name)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]
StandardScalerTransform(turned_on=False) turned_on
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]
MinMaxScalerTransform(turned_on=False) turned_on


In [63]:
np.random.choice([True, False])

True

In [30]:
val = [True, False]
val = [[1,2,3,4,5]]
print()
len([[1,2,3,4,5]])

True


1

In [5]:
import pandas as pd
import numpy as np
import shap
from pandasql import sqldf
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import load_boston

# Lambdas


def q(x):
    return sqldf(x, globals())

# a


def randomize_value_choice(value_options):
    value = None
    # Check to make sure there are value options
    if len(value_options > 0):
        # Make sure it isn't a 2D List
        if type(value_options[0]) != list:
            # Simple choice from the list - In the future this might include a distribution mean+std
            np.random.choice(value_options)
        else:
            # If the option is a 2D array it means we take each element with 50% probability
            value = []
            for element in value_options[0]:
                random_number = np.random.rand(1)[0]
                if random_number > 0.5:
                    value.append(element)
    return value


# My own version of RandomizedSearchCV
def RandomizedGridSearchCV(n_iter=10, pipe,
                           param_distributions,
                           scoring='neg_mean_squared_error', cv=2):

    # Transform the param_distributions into three arrays
    key_list = []
    transform_class_list = []
    parameter_name_list = []
    value_options_list = []
    for key, value_options in param_distributions.items():
        class_key, parameter_name = key.split("__")
        transform_class = pipe.named_steps[class_key]
        key_list.append(key)
        transform_class_list.append(transform_class)
        parameter_name_list.append(parameter_name)
        value_options_list.append(value_options)

    building_df = []
    for iteration in range(n_iter):
        inputs = {}
        # Updates the transform parameters
        for key, transform_class, parameter_name, value_options in zip(key_list, transform_class_list, parameter_name_list, value_options_list):

            # Get the random value
            value = randomize_value_choice(value_options)
            
            # Set the value
            setattr(class_list, parameter_name, value)
            
            # Record which inputs were chosen
            inputs[key] = value
            
        # Cross validation            
        scores = cross_val_score(
            pipe, train_X, train_y, scoring=scoring, cv=cv)
        
        # Average the scores
        score = np.mean(scores)
        
        #
       

    experiments_df = pd.DataFrame({'score': scores, 'fieldX': fieldX, 'fieldY': fieldY})
    return experiments_df

# Standard scaler data preparation class


class StandardScalerTransform(BaseEstimator, TransformerMixin):
    def __init__(self, turned_on=True):
        self.turned_on = turned_on
        self.standard_scaler = StandardScaler()

    def fit(self, X, y=None):
        if self.turned_on:
            self.standard_scaler.fit(X)
        return self

    def transform(self, X, y=None):
        if self.turned_on:
            X = self.standard_scaler.transform(X)
        return np.c_[X]


# Min-max scaler data preparation class


class MinMaxScalerTransform(BaseEstimator, TransformerMixin):
    def __init__(self, turned_on=True):
        self.turned_on = turned_on
        self.min_max_scaler = MinMaxScaler()

    def fit(self, X, y=None):
        if self.turned_on:
            self.min_max_scaler.fit(X)
        return self

    def transform(self, X, y=None):
        if self.turned_on:
            X = self.min_max_scaler.transform(X)
        return np.c_[X]


# # Standard scaler data preparation class

# class StandardScalerTransform(BaseEstimator, TransformerMixin):
#     def __init__(self, column_indices_to_replace=[]):
#         self.column_indices_to_replace = column_indices_to_replace
#         self.standard_scalers = {}

#     def fit(self, X, y=None):
#         for index in self.column_indices_to_replace:
#             self.standard_scalers[index] = StandardScaler()
#             self.standard_scalers[index].fit(X[:, index])
#         return self

#     def transform(self, X, y=None):
#         for index in self.column_indices_to_replace:
#             X[:, index] = self.standard_scalers[index].transform(X[:, index])
#         return np.c_[X]

# # Min-max scaler data preparation class

# class MinMaxScalerTransform(BaseEstimator, TransformerMixin):
#     def __init__(self, column_indices_to_replace=[]):
#         self.column_indices_to_replace = column_indices_to_replace
#         self.min_max_scalers = {}

#     def fit(self, X, y=None):
#         for index in self.column_indices_to_replace:
#             self.min_max_scalers[index] = MinMaxScaler()
#             self.min_max_scalers[index].fit(X[:, index])
#         return self

#     def transform(self, X, y=None):
#         for index in self.column_indices_to_replace:
#             X[:, index] = self.min_max_scalers[index].transform(X[:, index])
#         return np.c_[X]

# Generic xgboost fit using several grid searches


def get_xgboost_model(train_X, train_y):
    model = Pipeline([('xgb', XGBRegressor())])

    # 1) Tune max depth
    param_grid = [{
        'xgb__n_estimators': [100],
        'xgb__learning_rate': [0.1],
        'xgb__max_depth': [1, 2, 4, 6, 8],
        'xgb__subsample': [1.00]
    }]
    gs1 = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=2)
    gs1 = gs1.fit(train_X, train_y)
    max_depth = gs1.best_params_['xgb__max_depth']
    # print(gs1.best_score_)
    # print(gs1.best_params_)

    # 2) Tune subsample
    param_grid = [{
        'xgb__n_estimators': [100],
        'xgb__learning_rate': [0.1],
        'xgb__max_depth': [max_depth],
        'xgb__subsample': [0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1.00]
    }]
    gs2 = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=2)
    gs2 = gs2.fit(train_X, train_y)
    subsample = gs2.best_params_['xgb__subsample']
    # print(gs2.best_score_)
    # print(gs2.best_params_)

    # 3) Tune n_estimators
    param_grid = [{
        'xgb__n_estimators': [50, 100, 150, 200],
        'xgb__learning_rate': [0.1],
        'xgb__max_depth': [max_depth],
        'xgb__subsample': [subsample]
    }]
    gs3 = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=2)
    gs3 = gs3.fit(train_X, train_y)
    n_estimators = gs3.best_params_['xgb__n_estimators']
    # print(gs3.best_score_)
    # print(gs3.best_params_)

    # 4) Tune learning rate
    param_grid = [{
        'xgb__n_estimators': [n_estimators],
        'xgb__learning_rate': [0.1],
        'xgb__max_depth': [max_depth],
        'xgb__subsample': [subsample]
    },
        {
        'xgb__n_estimators': [n_estimators * 3],
        'xgb__learning_rate': [0.03],
        'xgb__max_depth': [max_depth],
        'xgb__subsample': [subsample]
    }]
    gs4 = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=2)
    gs4 = gs4.fit(train_X, train_y)
    n_estimators = gs4.best_params_['xgb__n_estimators']
    learning_rate = gs4.best_params_['xgb__learning_rate']
    # print(gs4.best_score_)
    # print(gs4.best_params_)

    # 5) Tune n_estimators
    param_grid = [{
        'xgb__n_estimators': [
            int(0.8 * n_estimators),
            int(1.0 * n_estimators),
            int(1.2 * n_estimators)
        ],
        'xgb__learning_rate': [learning_rate],
        'xgb__max_depth': [max_depth],
        'xgb__subsample': [subsample]
    }]
    gs5 = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=2)
    gs5 = gs5.fit(train_X, train_y)
    n_estimators = gs5.best_params_['xgb__n_estimators']
    # print(gs5.best_score_)
    # print(gs5.best_params_)

    # 6) Tune sampling by tree
    param_grid = [{
        'xgb__n_estimators': [n_estimators],
        'xgb__learning_rate': [learning_rate],
        'xgb__max_depth': [max_depth],
        'xgb__subsample': [subsample],
        'xgb__colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'xgb__colsample_bylevel': [1.0]
    }]
    gs6 = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=2)
    gs6 = gs6.fit(train_X, train_y)
    colsample_bytree = gs6.best_params_['xgb__colsample_bytree']
    # print(gs6.best_score_)
    # print(gs6.best_params_)

    # 7) Tune subsample
    param_grid = [{
        'xgb__n_estimators': [n_estimators],
        'xgb__learning_rate': [learning_rate],
        'xgb__max_depth': [max_depth],
        'xgb__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'xgb__colsample_bytree': [colsample_bytree],
        'xgb__colsample_bylevel': [1.0]
    }]
    gs7 = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=2)
    gs7 = gs7.fit(train_X, train_y)
    subsample = gs7.best_params_['xgb__subsample']
    # print(gs7.best_score_)
    # print(gs7.best_params_)

    # 8) Tune sampling by level
    n_estimators = gs7.best_params_['xgb__n_estimators']
    param_grid = [{
        'xgb__n_estimators': [n_estimators],
        'xgb__learning_rate': [learning_rate],
        'xgb__max_depth': [max_depth],
        'xgb__subsample': [subsample],
        'xgb__colsample_bytree': [colsample_bytree],
        'xgb__colsample_bylevel': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    }]
    gs8 = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=2)
    gs8 = gs8.fit(train_X, train_y)
    colsample_bylevel = gs8.best_params_['xgb__colsample_bylevel']
    # print(gs8.best_score_)
    # print(gs8.best_params_)

    # 9) Tune sampling fields
    n_estimators = gs8.best_params_['xgb__n_estimators']
    subsample = 0.9 if subsample == 1.0 else subsample
    colsample_bytree = 0.6 if colsample_bytree == 0.5 else colsample_bytree
    colsample_bylevel = 0.9 if colsample_bylevel == 1.0 else colsample_bylevel
    param_grid = [{
        'xgb__n_estimators': [n_estimators],
        'xgb__learning_rate': [learning_rate],
        'xgb__max_depth': [max_depth],
        'xgb__subsample': [subsample, subsample + 0.1],
        'xgb__colsample_bytree': [colsample_bytree - 0.1, colsample_bytree],
        'xgb__colsample_bylevel': [colsample_bylevel, colsample_bylevel + 0.1]
    }]
    gs9 = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=2)
    gs9 = gs9.fit(train_X, train_y)
    subsample = gs9.best_params_['xgb__subsample']
    colsample_bytree = gs9.best_params_['xgb__colsample_bytree']
    colsample_bylevel = gs9.best_params_['xgb__colsample_bylevel']
    # print(gs9.best_score_)
    # print(gs9.best_params_)

    # 10) Tune alpha
    param_grid = [{
        'xgb__n_estimators': [n_estimators],
        'xgb__learning_rate': [learning_rate],
        'xgb__max_depth': [max_depth],
        'xgb__subsample': [subsample],
        'xgb__colsample_bytree': [colsample_bytree],
        'xgb__colsample_bylevel': [colsample_bylevel],
        'xgb__reg_lambda': [0.001, 0.01, 0.1, 0.3, 1, 3, 10, 100, 1000]
    }]
    gs10 = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=2)
    gs10 = gs10.fit(train_X, train_y)
    # print(gs10.best_score_)
    # print(gs10.best_params_)

    # Find the best model
    # Sometimes the best model isn't the last one, so checking all of them
    best_model = gs1
    best_model_score = gs1.best_score_
    if gs2.best_score_ > best_model_score:
        best_model = gs2
        best_model_score = gs2.best_score_
    if gs2.best_score_ > best_model_score:
        best_model = gs2
        best_model_score = gs2.best_score_
    if gs3.best_score_ > best_model_score:
        best_model = gs3
        best_model_score = gs3.best_score_
    if gs4.best_score_ > best_model_score:
        best_model = gs4
        best_model_score = gs4.best_score_
    if gs5.best_score_ > best_model_score:
        best_model = gs5
        best_model_score = gs5.best_score_
    if gs6.best_score_ > best_model_score:
        best_model = gs6
        best_model_score = gs6.best_score_
    if gs7.best_score_ > best_model_score:
        best_model = gs7
        best_model_score = gs7.best_score_
    if gs8.best_score_ > best_model_score:
        best_model = gs8
        best_model_score = gs8.best_score_
    if gs9.best_score_ > best_model_score:
        best_model = gs9
        best_model_score = gs9.best_score_
    if gs10.best_score_ > best_model_score:
        best_model = gs10
        best_model_score = gs10.best_score_

    # Return the best model
    return XGBRegressor(**best_model.best_params_)

lightgbm is installed...but failed to load!


In [7]:
# Example dataset
boston_data = load_boston()

# Extract pandas dataframe and target
X = pd.DataFrame(boston_data['data'])
y = pd.DataFrame(boston_data['target'])

# Convert to numpy
train_X = X.values
train_y = y.values

# An okay model fit to the data

xgb_model = get_xgboost_model(train_X, train_y)

# Pipeline
pipe = Pipeline([('standard_scaler', StandardScalerTransform()),
                 ('min_max_scaler', MinMaxScalerTransform()),
                 ('model', xgb_model)])

# Find the number of features
num_features = train_X.shape[1]

# temp code
all_indices_to_consider = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
    
# Possible configurations
param_distributions = {
    'standard_scaler__column_indices_to_replace': [all_indices_to_consider],   
    'min_max_scaler__column_indices_to_replace': [all_indices_to_consider]
}

# Randomly search the space n_iter times
gs1 = RandomizedGridSearchCV(
    n_iter=10,
    pipe=pipe,
    param_distributions=param_distributions,
    scoring='neg_mean_squared_error',
    cv=2)
gs1 = gs1.fit(train_X, train_y)

# Get experiment data as a dataframe
cvres = gs1.cv_results_
experiments_df = pd.DataFrame.from_dict(cvres["params"])
experiments_df['score'] = cvres["mean_test_score"]
experiments_df.sort_values(by=['score'], ascending=False, inplace=True)

# Drop score
experiments_X_df = experiments_df.drop(['score'], axis=1)

# Get column names
X_column_names = experiments_X_df.columns

# Convert to numpy
experiments_X = experiments_X_df.values
experiments_y = experiments_df[['score']].values

# Create an XGBoost model tuned with the experiments data
xgb_experiments_model = get_xgboost_model(experiments_X, experiments_y)

# Fit the model
xgb_experiments_model.fit(experiments_X_df, experiments_y)

# Extract shap values
explainer = shap.TreeExplainer(xgb_experiments_model)
shap_values = explainer.shap_values(experiments_X_df)

# Shap as dataframe
pandas_shap_df = pd.DataFrame(shap_values, columns=X_column_names)
pandas_shap_df

/Users/petermyers/Desktop/Code/commons/venv/lib/python3.6/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


,min_max_scaler__turned_on,standard_scaler__turned_on
0,0.020893,0.0
1,0.020893,0.0
2,-0.020893,0.0
3,-0.020893,0.0


In [31]:
xgb_experiments_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1, xgb__colsample_bylevel=0.5,
       xgb__colsample_bytree=0.5, xgb__learning_rate=0.1, xgb__max_depth=1,
       xgb__n_estimators=240, xgb__reg_lambda=0.001, xgb__subsample=0.8)

In [29]:
cvres

/Users/petermyers/Desktop/Code/commons/venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/petermyers/Desktop/Code/commons/venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/petermyers/Desktop/Code/commons/venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.wa

{'mean_fit_time': array([0.00907099, 0.00820446, 0.00844753, 0.00824952]),
 'std_fit_time': array([5.37037849e-04, 8.34465027e-06, 1.56164169e-05, 1.65462494e-04]),
 'mean_score_time': array([0.00075603, 0.00070214, 0.00068033, 0.00063396]),
 'std_score_time': array([9.05990601e-06, 4.05311584e-06, 2.50339508e-06, 2.09808350e-05]),
 'param_standard_scaler__turned_on': masked_array(data=[True, False, True, False],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_max_scaler__turned_on': masked_array(data=[True, True, False, False],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_feature_selector__feature_index': masked_array(data=[0, 0, 0, 0],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'standard_scaler__turned_on': True,
   'min_max_scaler__turned_on': True,
   'feature_selector__feature_index':

In [9]:
experiments_df

,min_max_scaler__turned_on,standard_scaler__turned_on,score
0,True,True,-21.829335
1,True,False,-21.829335
2,False,True,-21.875476
3,False,False,-21.882115


In [10]:
# Transformation to polarized groups of shap values
polarized_shap_df = pandas_shap_df.copy()
for i in range(0, len(pandas_shap_df.index)):
    for j in range(0, len(pandas_shap_df.columns)):
        if not experiments_df.iloc[i, j]:
            polarized_df.iloc[i, j] = -1 * pandas_shap_df.iloc[i, j]
polarized_df

NameError: name 'polarized_df' is not defined

In [ ]:
# Certainly, I set a feature to True for large positive values
# Also, I set a feature to False for large negative values
# Otherwise, it is set to True or False
polarized_shap_result = polarized_df.sum()
polarized_shap_result